# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [85]:
## Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr, year, month, format_number
from pyspark.sql.types import (StructType, StructField, DateType,
                              DoubleType, IntegerType)

## Create Spark Session
spark = SparkSession.builder.appName('SparkProjExc').getOrCreate()

In [66]:
## Setup Schema
schema = StructType(fields=[StructField('date', DateType(), True),
                            StructField('open', DoubleType(), True),
                            StructField('high', DoubleType(), True),
                            StructField('low', DoubleType(), True),
                            StructField('close', DoubleType(), True),
                            StructField('volume', IntegerType(), True),
                            StructField('adj_close', DoubleType(), True)])

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [67]:
df = spark.read.csv('gs://spark-training-data/datasets/walmart_stock.csv',
                    inferSchema=False, header=True, schema=schema)

#### What are the column names?

In [68]:
df.columns

['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close']

#### What does the Schema look like?

In [69]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- adj_close: double (nullable = true)



#### Print out the first 5 columns.

In [70]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      date|              open|     high|      low|             close|  volume|         adj_close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [71]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              open|             high|              low|            close|           volume|        adj_close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [86]:
## Capture Output of Describe
result = df.describe()

In [87]:
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [72]:
hv_ratio = df.select([df['date'], df['high'] / df['volume']]).withColumnRenamed('(high / volume)', 'hv_ratio')
hv_ratio.show(5)

+----------+--------------------+
|      date|            hv_ratio|
+----------+--------------------+
|2012-01-03|4.819714653321546E-6|
|2012-01-04|6.290848613094555E-6|
|2012-01-05|4.669412994783916E-6|
|2012-01-06|7.367338463826307E-6|
|2012-01-09|8.915604778943901E-6|
+----------+--------------------+
only showing top 5 rows



#### What day had the Peak High in Price?

In [88]:
df.orderBy(df['high'].desc()).select(df['date']).show(1)

+----------+
|      date|
+----------+
|2015-01-13|
+----------+
only showing top 1 row



#### What is the mean of the Close column?

In [74]:
df.agg({'close': 'mean'}).show()

+-----------------+
|       avg(close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [75]:
df.agg({'volume': 'max'}).show()

+-----------+
|max(volume)|
+-----------+
|   80898100|
+-----------+



In [76]:
df.agg({'volume': 'min'}).show()

+-----------+
|min(volume)|
+-----------+
|    2094900|
+-----------+



#### How many days was the Close lower than 60 dollars?

In [77]:
df.filter(df['close'] < 60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [78]:
g80_result = df.filter(df['high'] > 80).count() / df.count() * 100
g80_result

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [79]:
df.select(corr(df['high'], df['volume'])).show()

+-------------------+
| corr(high, volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [82]:
df.select(['date', 'high']).groupby(year(df['date'])).max().show()

+----------+---------+
|year(date)|max(high)|
+----------+---------+
|      2015|90.970001|
|      2013|81.370003|
|      2014|88.089996|
|      2012|77.599998|
|      2016|75.190002|
+----------+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [89]:
df.select(['date', 'close']).groupby(month(df['date'])).mean().orderBy('month(date)').show()

+-----------+-----------------+
|month(date)|       avg(close)|
+-----------+-----------------+
|          1|71.44801958415842|
|          2|  71.306804443299|
|          3|71.77794377570092|
|          4|72.97361900952382|
|          5|72.30971688679247|
|          6| 72.4953774245283|
|          7|74.43971943925233|
|          8|73.02981855454546|
|          9|72.18411785294116|
|         10|71.57854545454543|
|         11| 72.1110893069307|
|         12|72.84792478301885|
+-----------+-----------------+



# Great Job!